In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import math
# import pixiedust                   ##debuger
# import glob, os

In [2]:
all_stim = pd.read_csv('2018_02_23_12_55_21_stim.csv', sep=';')

In [3]:
responses = pd.read_csv('2018_02_23_12_55_21.csv', sep=';', encoding = "ISO-8859-1")

# Feature task

In [4]:
#clean feature response with "bad response"
resp_feat = responses[responses["FeatConj"] == 0]
bad_feat = resp_feat.loc[(resp_feat['stType'] == ' distractor2 ') | (resp_feat['stType'] == ' distractor1 '), 
                         'trialN'].tolist()
resp_feat = resp_feat[~resp_feat['trialN']
          .isin(bad_feat)]
resp_feat = resp_feat.reset_index(drop=True)
resp_feat.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,col,row,touchX,touchY,touchDist,targDist,touchN,error,switches,changeScreen
0,11,33,m,0,0,b/y,0,40,50,0.0,...,4.0,6.0,467.5,565.5,0.0,0.000000,1,0,2,0
1,11,33,m,0,0,b/y,0,40,50,0.0,...,4.0,5.0,448.0,497.5,58.0538543078752,50.079188,2,0,0,0
2,11,33,m,0,0,b/y,0,40,50,0.0,...,3.0,5.0,385.5,505.5,58.4829034846937,71.467797,3,0,0,0
3,11,33,m,0,0,b/y,0,40,50,0.0,...,3.0,7.0,402.0,674.0,40.0499687890016,163.730622,6,0,1,0
4,11,33,m,0,0,b/y,0,40,50,0.0,...,1.0,3.0,236.0,311.5,396.927260338717,384.103581,7,0,0,0


In [5]:
resp_feat1 = resp_feat.rename(columns={'stType': 'type', 
                                          'stPosX': 'pos_x_r', 
                                          'stPosY': 'pos_y_r',
                                         'trialN': 'trial_r',
                                      'participantN': 'p_n'})
resp_feat1['pos_x_r'] = np.around(resp_feat1['pos_x_r'], decimals=2)
resp_feat1['pos_y_r'] = np.around(resp_feat1['pos_y_r'], decimals=2)
resp_feat['resp_pos'] = resp_feat1.apply(lambda x: list([x['pos_x_r'], x['pos_y_r']]) , axis=1)
resp_feat1['pos_r'] = list(zip(resp_feat1['pos_x_r'], resp_feat1['pos_y_r']))

resp_feat_c = resp_feat1.loc[:, resp_feat1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n',
                                                                'pos_x_r',
                                                                'pos_y_r'])]

resp_feat1 = resp_feat1.loc[:, resp_feat1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n'])]

resp_feat1 = resp_feat1.reset_index(drop=True)
resp_feat1.type = resp_feat1.type.str.strip()

In [6]:
resp_feat_c

,p_n,trial_r,type,pos_x_r,pos_y_r,pos_r
0,11,0,target1,462.66,564.97,"(462.66, 564.97)"
1,11,0,target1,455.46,515.41,"(455.46, 515.41)"
2,11,0,target1,384.59,506.15,"(384.59, 506.15)"
3,11,0,target2,406.85,668.36,"(406.85, 668.36)"
4,11,0,target2,236.61,324.05,"(236.61, 324.05)"
...,...,...,...,...,...,...
195,11,9,target2,385.35,642.07,"(385.35, 642.07)"
196,11,9,target2,483.64,646.00,"(483.64, 646.0)"
197,11,9,target1,719.11,567.01,"(719.11, 567.01)"
198,11,9,target2,889.77,668.22,"(889.77, 668.22)"


## Strict line funcs

In [7]:
def area(x1, y1, x2, y2, x3, y3): 
    
    return abs((x1 * (y2 - y3) + 
                x2 * (y3 - y1) + 
                x3 * (y1 - y2)) / 2.0)

In [8]:
def check_area(x1, y1, x2, y2, lst1, lst2):
    r = 10
    e = -1
    #define lines coefficients
    m = (y1 - y2)/(x1-x2)
    b = y1 - m*x1
    if m != 0:
        m_c = e/m
    else:
        m_c = 0
    b1 = y1 - m_c*x1
    b2 = y2 - m_c*x2
    #define check points
    R = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    kx = (r/R) * (y2 - y1)
    ky = (r/R) * (x2 - x1)
    x1_check1 = x1 - kx
    y1_check1 = y1 + ky
    x1_check2 = x1 + kx
    y1_check2 = y1 - ky
    x2_check1 = x2 - kx
    y2_check1 = y2 + ky
    x2_check2 = x2 + kx
    y2_check2 = y2 - ky
    #check if the point inside the rectangle
    final_list= []
    for i in range(0, len(lst1)):#area of triangles and rectangle
        s1 = area(lst1[i], lst2[i], x1_check1, y1_check1, x1_check2, y1_check2)
        s2 = area(lst1[i], lst2[i], x1_check2, y1_check2, x2_check2, y2_check2)
        s3 = area(lst1[i], lst2[i], x2_check2, y2_check2, x2_check1, y2_check1)
        s4 = area(lst1[i], lst2[i], x2_check1, y2_check1, x1_check1, y1_check1)
        s_tr = round((s1 + s2 + s3 + s4), 3)
        s_rec = round((r * 2 * R), 3)
        #final check
        if s_tr == s_rec:
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list  
            

In [9]:
check_area(0, 0, 1, 1, [0,0], [0.5,2])

[1, 1]

In [10]:
len([5,3,3])

3

### Add x1,x2,y1,y2

In [11]:
resp_feat_c['x1'] = resp_feat_c['pos_x_r'].apply(lambda x: x + 108.6)
resp_feat_c['x2'] = resp_feat_c['pos_x_r'].apply(lambda x: x - 108.6)
resp_feat_c['y1'] = resp_feat_c['pos_y_r'].apply(lambda x: x + 106.5)
resp_feat_c['y2'] = resp_feat_c['pos_y_r'].apply(lambda x: x - 106.5)

In [12]:
resp_feat_c

,p_n,trial_r,type,pos_x_r,pos_y_r,pos_r,x1,x2,y1,y2
0,11,0,target1,462.66,564.97,"(462.66, 564.97)",571.26,354.06,671.47,458.47
1,11,0,target1,455.46,515.41,"(455.46, 515.41)",564.06,346.86,621.91,408.91
2,11,0,target1,384.59,506.15,"(384.59, 506.15)",493.19,275.99,612.65,399.65
3,11,0,target2,406.85,668.36,"(406.85, 668.36)",515.45,298.25,774.86,561.86
4,11,0,target2,236.61,324.05,"(236.61, 324.05)",345.21,128.01,430.55,217.55
...,...,...,...,...,...,...,...,...,...,...
195,11,9,target2,385.35,642.07,"(385.35, 642.07)",493.95,276.75,748.57,535.57
196,11,9,target2,483.64,646.00,"(483.64, 646.0)",592.24,375.04,752.50,539.50
197,11,9,target1,719.11,567.01,"(719.11, 567.01)",827.71,610.51,673.51,460.51
198,11,9,target2,889.77,668.22,"(889.77, 668.22)",998.37,781.17,774.72,561.72


### Distances

In [13]:
feat_r = resp_feat1.values.tolist()

In [14]:
#only useful df of all stimuli for feature task
all_feat = all_stim[all_stim["FeatConj"] == 0]
feat_target = all_feat[(all_feat['stType'] == 'target1') 
                       | (all_feat['stType'] == 'target2')]
feat_distr = all_feat[(all_feat['stType'] == 'distractor1') 
                      | (all_feat['stType'] == 'distractor2')]

In [15]:
#useful targets for feature (only 3 columns)
feat_rest_t = feat_target[~feat_target['trialN']
          .isin(bad_feat)]
feat_rest_t = feat_rest_t.rename(columns={'stType': 'target', 
                                          'posX': 'pos_x_t', 
                                          'posY': 'pos_y_t',
                                         'trialN': 'trial_t',
                                         'participantN': 'p_n_t'})
feat_rest_t['pos_x_t'] = np.around(feat_rest_t['pos_x_t'], decimals=2)
feat_rest_t['pos_y_t'] = np.around(feat_rest_t['pos_y_t'], decimals=2)
feat_rest_t['pos_t'] = list(zip(feat_rest_t['pos_x_t'], feat_rest_t['pos_y_t']))
feat_rest_t = feat_rest_t.loc[:, feat_rest_t.columns.intersection(['target',
                                                                   'pos_t',
                                                                   'trial_t',
                                                                  'p_n_t'])]
feat_rest_t = feat_rest_t.reset_index(drop=True)

In [16]:
#numbers of trials in list
trials = feat_rest_t['trial_t'].unique()
trials = trials.astype(np.int32).tolist()

In [17]:
participant = feat_rest_t['p_n_t'].unique()
participant = participant.astype(np.int32).tolist()

In [18]:
dfs_tar_f = dict(tuple(feat_rest_t.groupby('target')))
feat_t1 = dfs_tar_f['target1']
feat_t2 = dfs_tar_f['target2']

In [19]:
t1 = dict(tuple(feat_t1.groupby('trial_t')))
t2 = dict(tuple(feat_t2.groupby('trial_t')))

In [20]:
def closest_node(node, nodes):
    indexArr = np.argwhere(nodes == node)
    nodes = np.delete(nodes,indexArr, 0)
    closest_index = distance.cdist([node], nodes, 'euclidean').argmin()
    return nodes[closest_index]

In [21]:
# %%pixie_debugger 
closest_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1':
                        closest_t1 = closest_node(feat_r[j][3], t1_arr)                          
                        closest_same.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(feat_r[j][3], t2_arr)                         
                        closest_same.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1
               

In [22]:
resp_feat['closest_same_target'] = closest_same

In [23]:
closest_other = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target2':
                        closest_t1 = closest_node(feat_r[j][3], t1_arr)                          
                        closest_other.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(feat_r[j][3], t2_arr)                         
                        closest_other.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [24]:
resp_feat['closest_other_taget'] = closest_other

In [25]:
feat_rest_d = feat_distr[~feat_distr['trialN']
          .isin(bad_feat)]
feat_rest_d = feat_rest_d.rename(columns={'stType': 'distractor', 
                                          'posX': 'pos_x_d', 
                                          'posY': 'pos_y_d',
                                         'trialN': 'trial_d',
                                         'participantN': 'p_n_d'})
feat_rest_d['pos_x_d'] = np.around(feat_rest_d['pos_x_d'], decimals=2)
feat_rest_d['pos_y_d'] = np.around(feat_rest_d['pos_y_d'], decimals=2)
feat_rest_d['pos_d'] = list(zip(feat_rest_d['pos_x_d'], feat_rest_d['pos_y_d']))
feat_rest_d = feat_rest_d.loc[:, feat_rest_d.columns.intersection(['distractor',
                                                                   'pos_d',
                                                                   'trial_d',
                                                                  'p_n_d'])]
feat_rest_d = feat_rest_d.reset_index(drop=True)

In [26]:
dfs_d_f = dict(tuple(feat_rest_d.groupby('distractor')))
feat_d1 = dfs_d_f['distractor1']
feat_d2 = dfs_d_f['distractor2']
d1 = dict(tuple(feat_d1.groupby('trial_d')))
d2 = dict(tuple(feat_d2.groupby('trial_d')))

In [27]:
# closest blue dot
closest_same_d = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        d1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    closest_d1 = closest_node(feat_r[j][3], d1_arr)                          
                    closest_same_d.append(closest_d1.tolist())         
                else:
                    i += 1
            else:
                n += 1

In [28]:
resp_feat['closest_blue_dot'] = closest_same_d

In [29]:
# closest yellow dot
closest_d2 = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        d1_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    c_d1 = closest_node(feat_r[j][3], d1_arr)                          
                    closest_d2.append(c_d1.tolist())         
                else:
                    i += 1
            else:
                n += 1

In [30]:
resp_feat['closest_yellow_dot'] = closest_d2

In [31]:
resp_feat_c

,p_n,trial_r,type,pos_x_r,pos_y_r,pos_r,x1,x2,y1,y2
0,11,0,target1,462.66,564.97,"(462.66, 564.97)",571.26,354.06,671.47,458.47
1,11,0,target1,455.46,515.41,"(455.46, 515.41)",564.06,346.86,621.91,408.91
2,11,0,target1,384.59,506.15,"(384.59, 506.15)",493.19,275.99,612.65,399.65
3,11,0,target2,406.85,668.36,"(406.85, 668.36)",515.45,298.25,774.86,561.86
4,11,0,target2,236.61,324.05,"(236.61, 324.05)",345.21,128.01,430.55,217.55
...,...,...,...,...,...,...,...,...,...,...
195,11,9,target2,385.35,642.07,"(385.35, 642.07)",493.95,276.75,748.57,535.57
196,11,9,target2,483.64,646.00,"(483.64, 646.0)",592.24,375.04,752.50,539.50
197,11,9,target1,719.11,567.01,"(719.11, 567.01)",827.71,610.51,673.51,460.51
198,11,9,target2,889.77,668.22,"(889.77, 668.22)",998.37,781.17,774.72,561.72


# Crowding_feat

In [32]:
def close_or_not(x1, x2, y1, y2, lst1, lst2):
    count = 0
    for i in range(0, len(lst1)):
        if x2 < lst1[i] < x1 and y2 < lst2[i] < y1:
            count += 1
    return count

In [33]:
feat_r = resp_feat_c.values.tolist()

In [34]:
feat_t1[['pos_x', 'pos_y']] = pd.DataFrame(feat_t1['pos_t'].tolist(), index=feat_t1.index) 
feat_t2[['pos_x', 'pos_y']] = pd.DataFrame(feat_t1['pos_t'].tolist(), index=feat_t1.index)

In [35]:
t1 = dict(tuple(feat_t1.groupby('trial_t')))
t2 = dict(tuple(feat_t2.groupby('trial_t')))

In [36]:
crowding = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1':
                        closest_t1 = close_or_not(feat_r[j][6],
                                                  feat_r[j][7],
                                                  feat_r[j][8],
                                                  feat_r[j][9],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not(feat_r[j][6],
                                                  feat_r[j][7],
                                                  feat_r[j][8],
                                                  feat_r[j][9],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [37]:
resp_feat['crowd'] = crowding

## Feature strict line

In [38]:
all_feat_sl = all_feat.loc[:, all_feat.columns.intersection(['participantN',
                                                             'trialN',
                                                         'stType',
                                                         'posX',
                                                         'posY'])]

In [39]:
all_feat_sl

,participantN,trialN,stType,posX,posY
520,11,0,target1,455.457126,515.407551
521,11,0,target1,384.591500,506.149626
522,11,0,target1,462.660016,564.966037
523,11,0,target1,802.373050,658.384035
524,11,0,target1,870.350992,214.984996
...,...,...,...,...,...
915,11,9,distractor2,120.173545,215.477899
916,11,9,distractor2,721.610711,642.845646
917,11,9,distractor2,275.078947,247.519506
918,11,9,distractor2,395.258517,276.110686


In [40]:
grouped_all_feat = dict(tuple(all_feat_sl.groupby('trialN')))

In [41]:
resp_feat_sl = resp_feat.loc[:, resp_feat.columns.intersection(['participantN',
                                                                'trialN',
                                                                'stType',
                                                                'resp_pos',
                                                                'closest_same_target'])]

In [42]:
resp_feat_sl[['cl_x', 'cl_y']] = pd.DataFrame(resp_feat_sl['closest_same_target']
                                              .tolist(), index=resp_feat_sl.index) 
resp_feat_sl[['r_x', 'r_r']] = pd.DataFrame(resp_feat_sl['resp_pos']
                                              .tolist(), index=resp_feat_sl.index) 

In [43]:
resp_feat_sl

,participantN,trialN,stType,resp_pos,closest_same_target,cl_x,cl_y,r_x,r_r
0,11,0,target1,"[462.66, 564.97]","[743.29, 422.43]",743.29,422.43,462.66,564.97
1,11,0,target1,"[455.46, 515.41]","[462.66, 564.97]",462.66,564.97,455.46,515.41
2,11,0,target1,"[384.59, 506.15]","[462.66, 564.97]",462.66,564.97,384.59,506.15
3,11,0,target2,"[406.85, 668.36]","[660.24, 367.13]",660.24,367.13,406.85,668.36
4,11,0,target2,"[236.61, 324.05]","[397.95, 108.04]",397.95,108.04,236.61,324.05
...,...,...,...,...,...,...,...,...,...
195,11,9,target2,"[385.35, 642.07]","[483.64, 646.0]",483.64,646.00,385.35,642.07
196,11,9,target2,"[483.64, 646.0]","[275.09, 648.33]",275.09,648.33,483.64,646.00
197,11,9,target1,"[719.11, 567.01]","[705.85, 346.7]",705.85,346.70,719.11,567.01
198,11,9,target2,"[889.77, 668.22]","[890.15, 432.35]",890.15,432.35,889.77,668.22


In [44]:
trials = all_feat_sl['trialN'].unique()
trials = trials.astype(np.int32).tolist()

In [45]:
feat_r_sl = resp_feat_sl.values.tolist()

In [46]:
strict_line = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        arr_x = grouped_all_feat.get(trials[i])['posX'].values.tolist()
        arr_y = grouped_all_feat.get(trials[i])['posY'].values.tolist()
        
        for j in range(0, len(feat_r_sl)):
            if part == feat_r_sl[j][0]:
                if n_tr == feat_r_sl[j][1]:
                    strict = check_area(feat_r_sl[j][7], 
                                        feat_r_sl[j][8],
                                        feat_r_sl[j][5], 
                                        feat_r_sl[j][6],
                                        arr_x, arr_y)
                    strict_line.append(strict)
            else:
                i += 1
        else:
            n += 1

In [47]:
can_provide_line = []
for i in range(0, len(strict_line)):
        if 1 in strict_line[i]:
            can_provide_line.append(1)
        else:
            can_provide_line.append(0)
    

In [48]:
resp_feat['strict_line'] = can_provide_line

# Conjunction task

In [49]:
resp_conj = responses[responses["FeatConj"] == 1]
bad_conj = resp_conj.loc[(resp_conj['stType'] == ' distractor2 ') | (resp_conj['stType'] == ' distractor1 '), 
                         'trialN'].tolist()
resp_conj = resp_conj[~resp_conj['trialN']
          .isin(bad_conj)]
resp_conj = resp_conj.reset_index(drop=True)

In [50]:
resp_conj1 = resp_conj.rename(columns={'stType': 'type', 
                                          'stPosX': 'pos_x_r', 
                                          'stPosY': 'pos_y_r',
                                         'trialN': 'trial_r',
                                      'participantN': 'p_n'})
resp_conj1['pos_x_r'] = np.around(resp_conj1['pos_x_r'], decimals=2)
resp_conj1['pos_y_r'] = np.around(resp_conj1['pos_y_r'], decimals=2)
resp_conj['resp_pos'] = resp_conj1.apply(lambda x: list([x['pos_x_r'], x['pos_y_r']]) , axis=1)
resp_conj1['pos_r'] = list(zip(resp_conj1['pos_x_r'], resp_conj1['pos_y_r']))

resp_conj_c = resp_conj1.loc[:, resp_conj1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n',
                                                               'pos_x_r',
                                                               'pos_y_r'])]

resp_conj1 = resp_conj1.loc[:, resp_conj1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n'])]

resp_conj1 = resp_conj1.reset_index(drop=True)
resp_conj1.type = resp_conj1.type.str.strip()

### Add x1,x2,y1,y2

In [51]:
resp_conj_c['x1'] = resp_conj_c['pos_x_r'].apply(lambda x: x + 108.6)
resp_conj_c['x2'] = resp_conj_c['pos_x_r'].apply(lambda x: x - 108.6)
resp_conj_c['y1'] = resp_conj_c['pos_y_r'].apply(lambda x: x + 106.5)
resp_conj_c['y2'] = resp_conj_c['pos_y_r'].apply(lambda x: x - 106.5)

### Distances

In [52]:
conj_r = resp_conj1.values.tolist()
## [prtN, trialN, target_type, pos]

In [53]:
#only useful df of all stimuli for conjunction task
all_conj = all_stim[all_stim["FeatConj"] == 1]
conj_target = all_conj[(all_conj['stType'] == 'target1') |
                       (all_conj['stType'] == 'target2')]
conj_distr = all_conj[(all_conj['stType'] == 'distractor1') |
                      (all_conj['stType'] == 'distractor2')]

In [54]:
#useful targets for conjunction
conj_rest_t = conj_target[~conj_target['trialN']
          .isin(bad_conj)]
conj_rest_t = conj_rest_t.rename(columns={'stType': 'target', 
                                          'posX': 'pos_x_t', 
                                          'posY': 'pos_y_t',
                                         'trialN': 'trial_t',
                                         'participantN': 'p_n_t'})
conj_rest_t['pos_x_t'] = np.around(conj_rest_t['pos_x_t'], decimals=2)
conj_rest_t['pos_y_t'] = np.around(conj_rest_t['pos_y_t'], decimals=2)
conj_rest_t['pos_t'] = list(zip(conj_rest_t['pos_x_t'], conj_rest_t['pos_y_t']))
conj_rest_t = conj_rest_t.loc[:, conj_rest_t.columns.intersection(['target',
                                                                   'pos_t',
                                                                   'trial_t',
                                                                  'p_n_t'])]
conj_rest_t = conj_rest_t.reset_index(drop=True)

In [55]:
trials = conj_rest_t['trial_t'].unique()
trials = trials.astype(np.int32).tolist()

In [56]:
participant = conj_rest_t['p_n_t'].unique()
participant = participant.astype(np.int32).tolist()

In [57]:
dfs_tar_c = dict(tuple(conj_rest_t.groupby('target')))
conj_t1 = dfs_tar_c['target1']
conj_t2 = dfs_tar_c['target2']

In [58]:
t1 = dict(tuple(conj_t1.groupby('trial_t')))
t2 = dict(tuple(conj_t2.groupby('trial_t')))

In [59]:
closest_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        closest_same.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        closest_same.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [60]:
resp_conj['closest_same_target'] = closest_same

In [61]:
closest_other = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target2':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        closest_other.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        closest_other.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [62]:
resp_conj['closest_other_target'] = closest_other

In [63]:
#useful distractors for conjunction
conj_rest_d = conj_distr[~conj_distr['trialN']
          .isin(bad_conj)]
conj_rest_d = conj_rest_d.rename(columns={'stType': 'distractor', 
                                          'posX': 'pos_x_d', 
                                          'posY': 'pos_y_d',
                                         'trialN': 'trial_d',
                                         'participantN': 'p_n_d'})
conj_rest_d['pos_x_d'] = np.around(conj_rest_d['pos_x_d'], decimals=2)
conj_rest_d['pos_y_d'] = np.around(conj_rest_d['pos_y_d'], decimals=2)
conj_rest_d['pos_d'] = list(zip(conj_rest_d['pos_x_d'], conj_rest_d['pos_y_d']))
conj_rest_d = conj_rest_d.loc[:, conj_rest_d.columns.intersection(['distractor',
                                                                   'pos_d',
                                                                   'trial_d',
                                                                  'p_n_d'])]
conj_rest_d = conj_rest_d.reset_index(drop=True)

In [64]:
dfs_d_c = dict(tuple(conj_rest_d.groupby('distractor')))
conj_d1 = dfs_d_c['distractor1']
conj_d2 = dfs_d_c['distractor2']
d1 = dict(tuple(conj_d1.groupby('trial_d')))
d2 = dict(tuple(conj_d2.groupby('trial_d')))

In [65]:
same_color = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        t2_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_d1 = closest_node(conj_r[j][3], t1_arr)                          
                        same_color.append(closest_d1.tolist())         
                    else:
                        closest_d2 = closest_node(conj_r[j][3], t2_arr)                         
                        same_color.append(closest_d2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [66]:
resp_conj['closest_dist_same_color'] = same_color

In [67]:
same_shape = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        t2_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target2':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        same_shape.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        same_shape.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [68]:
resp_conj['closest_dist_same_shape'] = same_shape

# Crowding_conj

In [69]:
conj_r = resp_conj_c.values.tolist()

In [70]:
conj_t1[['pos_x', 'pos_y']] = pd.DataFrame(conj_t1['pos_t'].tolist(), index=conj_t1.index) 
conj_t2[['pos_x', 'pos_y']] = pd.DataFrame(conj_t2['pos_t'].tolist(), index=conj_t1.index)

In [71]:
t1 = dict(tuple(conj_t1.groupby('trial_t')))
t2 = dict(tuple(conj_t2.groupby('trial_t')))

In [72]:
crowding = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_t1 = close_or_not(conj_r[j][6],
                                                  conj_r[j][7],
                                                  conj_r[j][8],
                                                  conj_r[j][9],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not(conj_r[j][6],
                                                  conj_r[j][7],
                                                  conj_r[j][8],
                                                  conj_r[j][9],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [73]:
resp_conj['crowd'] = crowding

## Strict line conj

In [74]:
all_conj_sl = all_conj.loc[:, all_conj.columns.intersection(['participantN',
                                                             'trialN',
                                                         'stType',
                                                         'posX',
                                                         'posY'])]

In [75]:
all_conj_sl

,participantN,trialN,stType,posX,posY
0,11,0,target1,721.567646,519.221115
1,11,0,target1,405.731109,511.071306
2,11,0,target1,917.367805,425.130014
3,11,0,target1,390.398777,675.190748
4,11,0,target1,396.962515,421.843652
...,...,...,...,...,...
515,11,12,distractor2,731.112533,507.419280
516,11,12,distractor2,647.276979,591.815156
517,11,12,distractor2,822.711772,510.440555
518,11,12,distractor2,286.565521,568.302168


In [76]:
grouped_all_conj = dict(tuple(all_conj_sl.groupby('trialN')))

In [77]:
resp_conj_sl = resp_conj.loc[:, resp_conj.columns.intersection(['participantN',
                                                                'trialN',
                                                                'stType',
                                                                'resp_pos',
                                                                'closest_same_target'])]

In [78]:
resp_conj_sl[['cl_x', 'cl_y']] = pd.DataFrame(resp_conj_sl['closest_same_target']
                                              .tolist(), index=resp_conj_sl.index) 
resp_conj_sl[['r_x', 'r_y']] = pd.DataFrame(resp_conj_sl['resp_pos']
                                              .tolist(), index=resp_conj_sl.index) 

In [79]:
resp_conj_sl

,participantN,trialN,stType,resp_pos,closest_same_target,cl_x,cl_y,r_x,r_y
0,11,0,target1,"[917.37, 425.13]","[909.24, 289.23]",909.24,289.23,917.37,425.13
1,11,0,target2,"[890.16, 325.21]","[576.15, 521.5]",576.15,521.50,890.16,325.21
2,11,0,target1,"[909.24, 289.23]","[917.37, 425.13]",917.37,425.13,909.24,289.23
3,11,0,target2,"[195.09, 514.3]","[102.71, 483.94]",102.71,483.94,195.09,514.30
4,11,0,target2,"[102.71, 483.94]","[195.09, 514.3]",195.09,514.30,102.71,483.94
...,...,...,...,...,...,...,...,...,...
213,11,12,target2,"[627.16, 438.5]","[900.71, 507.81]",900.71,507.81,627.16,438.50
214,11,12,target2,"[621.53, 362.26]","[627.16, 438.5]",627.16,438.50,621.53,362.26
215,11,12,target1,"[910.49, 430.44]","[543.24, 403.99]",543.24,403.99,910.49,430.44
216,11,12,target1,"[888.22, 214.9]","[910.49, 430.44]",910.49,430.44,888.22,214.90


In [80]:
trials = all_conj_sl['trialN'].unique()
trials = trials.astype(np.int32).tolist()

In [81]:
conj_r_sl = resp_conj_sl.values.tolist()

In [82]:
conj_r_sl

[[11,
  0,
  'target1',
  [917.37, 425.13],
  [909.24, 289.23],
  909.24,
  289.23,
  917.37,
  425.13],
 [11,
  0,
  'target2',
  [890.16, 325.21],
  [576.15, 521.5],
  576.15,
  521.5,
  890.16,
  325.21],
 [11,
  0,
  'target1',
  [909.24, 289.23],
  [917.37, 425.13],
  917.37,
  425.13,
  909.24,
  289.23],
 [11,
  0,
  'target2',
  [195.09, 514.3],
  [102.71, 483.94],
  102.71,
  483.94,
  195.09,
  514.3],
 [11,
  0,
  'target2',
  [102.71, 483.94],
  [195.09, 514.3],
  195.09,
  514.3,
  102.71,
  483.94],
 [11,
  0,
  'target2',
  [192.46, 366.53],
  [102.71, 483.94],
  102.71,
  483.94,
  192.46,
  366.53],
 [11,
  0,
  'target1',
  [310.11, 350.08],
  [396.96, 421.84],
  396.96,
  421.84,
  310.11,
  350.08],
 [11,
  0,
  'target1',
  [396.96, 421.84],
  [466.87, 479.67],
  466.87,
  479.67,
  396.96,
  421.84],
 [11,
  0,
  'target1',
  [466.87, 479.67],
  [396.96, 421.84],
  396.96,
  421.84,
  466.87,
  479.67],
 [11,
  0,
  'target1',
  [405.73, 511.07],
  [466.87, 479.67

In [83]:
strict_line = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        arr_x = grouped_all_conj.get(trials[i])['posX'].values.tolist()
        arr_y = grouped_all_conj.get(trials[i])['posY'].values.tolist()
        
        for j in range(0, len(conj_r_sl)):
            if part == conj_r_sl[j][0]:
                if n_tr == conj_r_sl[j][1]:
                    strict = check_area(conj_r_sl[j][7], 
                                        conj_r_sl[j][8],
                                        conj_r_sl[j][5], 
                                        conj_r_sl[j][6],
                                        arr_x, arr_y)
                    strict_line.append(strict)
            else:
                i += 1
        else:
            n += 1

In [84]:
len(strict_line)

218

In [85]:
can_provide_line = []
for i in range(0, len(strict_line)):
    if 1 in strict_line[i]:
        can_provide_line.append(1)
    else:
        can_provide_line.append(0)
    

In [86]:
resp_conj['strict_line'] = can_provide_line

# Make csv

In [87]:
resp_feat.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,error,switches,changeScreen,resp_pos,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,crowd,strict_line
0,11,33,m,0,0,b/y,0,40,50,0.0,...,0,2,0,"[462.66, 564.97]","[743.29, 422.43]","[406.85, 668.36]","[543.6, 500.52]","[297.59, 488.77]",3,1
1,11,33,m,0,0,b/y,0,40,50,0.0,...,0,0,0,"[455.46, 515.41]","[462.66, 564.97]","[406.85, 668.36]","[543.6, 500.52]","[297.59, 488.77]",3,1
2,11,33,m,0,0,b/y,0,40,50,0.0,...,0,0,0,"[384.59, 506.15]","[462.66, 564.97]","[406.85, 668.36]","[543.6, 500.52]","[297.59, 488.77]",3,1
3,11,33,m,0,0,b/y,0,40,50,0.0,...,0,1,0,"[406.85, 668.36]","[660.24, 367.13]","[462.66, 564.97]","[230.73, 585.57]","[223.65, 638.27]",0,1
4,11,33,m,0,0,b/y,0,40,50,0.0,...,0,0,0,"[236.61, 324.05]","[397.95, 108.04]","[384.59, 506.15]","[116.04, 329.06]","[304.95, 359.1]",0,1


In [88]:
resp_feat.columns

Index(['participantN', 'age', 'gender', 'trialN', 'task', 'condition',
       'FeatConj', 'setSize', 'proportion', 'timelimit', 'stType', 'imgName',
       'timeTS', 'timeRel', 'runLength', 'runTest', 'touchTS', 'runN', 'runNH',
       'stPosX', 'stPosY', 'col', 'row', 'touchX', 'touchY', 'touchDist',
       'targDist', 'touchN', 'error', 'switches', 'changeScreen', 'resp_pos',
       'closest_same_target', 'closest_other_taget', 'closest_blue_dot',
       'closest_yellow_dot', 'crowd', 'strict_line'],
      dtype='object')

In [89]:
resp_conj

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,error,switches,changeScreen,resp_pos,closest_same_target,closest_other_target,closest_dist_same_color,closest_dist_same_shape,crowd,strict_line
0,11,33,m,0,0,rs/gd,1,40,50,0.0,...,0,2,0,"[917.37, 425.13]","[909.24, 289.23]","[890.16, 325.21]","[817.61, 362.76]","[793.07, 436.27]",1,1
1,11,33,m,0,0,rs/gd,1,40,50,0.0,...,0,1,0,"[890.16, 325.21]","[576.15, 521.5]","[909.24, 289.23]","[793.07, 436.27]","[817.61, 362.76]",0,1
2,11,33,m,0,0,rs/gd,1,40,50,0.0,...,0,1,0,"[909.24, 289.23]","[917.37, 425.13]","[890.16, 325.21]","[817.61, 362.76]","[730.77, 289.65]",1,1
3,11,33,m,0,0,rs/gd,1,40,50,0.0,...,0,1,0,"[195.09, 514.3]","[102.71, 483.94]","[310.11, 350.08]","[318.54, 197.06]","[293.16, 263.84]",0,1
4,11,33,m,0,0,rs/gd,1,40,50,0.0,...,0,0,0,"[102.71, 483.94]","[195.09, 514.3]","[310.11, 350.08]","[318.54, 197.06]","[293.16, 263.84]",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,11,33,m,12,0,rs/gd,1,40,50,0.0,...,0,0,0,"[627.16, 438.5]","[900.71, 507.81]","[543.24, 403.99]","[731.11, 507.42]","[792.88, 401.91]",0,0
214,11,33,m,12,0,rs/gd,1,40,50,0.0,...,0,0,0,"[621.53, 362.26]","[627.16, 438.5]","[543.24, 403.99]","[488.36, 278.73]","[579.86, 198.01]",0,0
215,11,33,m,12,0,rs/gd,1,40,50,0.0,...,0,1,0,"[910.49, 430.44]","[543.24, 403.99]","[900.71, 507.81]","[792.88, 401.91]","[822.71, 510.44]",1,1
216,11,33,m,12,0,rs/gd,1,40,50,0.0,...,0,0,0,"[888.22, 214.9]","[910.49, 430.44]","[706.61, 125.4]","[910.98, 289.61]","[822.71, 510.44]",1,1


In [90]:
resp_feat.to_csv (r'C:\Users\lttso\Desktop\feature_task.csv', index = False,  mode='a', header=False)

In [91]:
resp_conj.to_csv (r'C:\Users\lttso\Desktop\conjunction_task.csv', index = False, mode='a', header=False)